In [4]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time
import requests
import re

In [76]:
from nba_api.stats.static import teams as nba_teams_module
from nba_api.stats.endpoints.commonteamroster import CommonTeamRoster
from nba_api.stats.endpoints.leaguestandingsv3 import LeagueStandingsV3
from nba_api.stats.endpoints.playercompare import PlayerCompare
from nba_api.stats.endpoints import playercareerstats, playergamelog
from nba_api.stats.endpoints import winprobabilitypbp

In [74]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)

# Create Team and Roster Class

In [93]:
class NBATeamRosters:
    def __init__(self, season):
        """
        Initializes the class and sets up the teams DataFrame and an empty dictionary for rosters.
        Args:
            season (str): The NBA season in 'YYYY' format, e.g., '2024' for the 2024-2025 season.
        """
        self.season = season
        self.teams_df = None
        self.rosters = {}
        self.standings_df = None
        self.schedule_df = None
        self.player_stats = {}
        self.fetch_teams()

    def fetch_teams(self):
        """
        Fetches NBA teams and stores them in a DataFrame.
        """
        nba_teams = nba_teams_module.get_teams()
        self.teams_df = pd.DataFrame(nba_teams).set_index('id')

    def fetch_rosters(self):
        """
        Fetches and stores the rosters for each team.
        The rosters are stored in a dictionary with keys as the team's abbreviation, nickname, and city.
        """
        for team_id, team_data in self.teams_df.iterrows():
            team_roster = self.get_team_roster(team_id)
            if team_roster is not None:
                self.rosters[team_data['abbreviation']] = team_roster
                self.rosters[team_data['nickname']] = team_roster
                self.rosters[team_data['city']] = team_roster

    def get_team_roster(self, team_id):
        """
        Fetches the roster for a single team based on the team ID.
        Args:
            team_id (int): The team's ID.

        Returns:
            pd.DataFrame: DataFrame containing the team's roster.
        """
        try:
            rosters_raw = CommonTeamRoster(
                team_id=str(team_id),
                season=self.season
            ).common_team_roster.get_data_frame()
            return rosters_raw
        except Exception as e:
            print(f"Error fetching roster for team ID {team_id}: {e}")
            return None

    def get_roster_by_key(self, key):
        """
        Retrieves the roster DataFrame by the team's abbreviation, nickname, or city.
        Args:
            key (str): The team's abbreviation, nickname, or city.

        Returns:
            pd.DataFrame: DataFrame containing the team's roster.
        """
        return self.rosters.get(key, None)

    def load_schedule(self, csv_path):
        """
        Loads the NBA schedule from a CSV file into the class.
        Args:
            csv_path (str): The file path to the schedule CSV file.
        """
        self.schedule_df = pd.read_csv(csv_path)
        print(f"Schedule loaded: {len(self.schedule_df)} games")

    def get_teams_for_game(self, game_id):
        """
        Retrieves the teams playing in a specific game from the schedule.
        Args:
            game_id (str): The ID of the game.
    
        Returns:
            Tuple[pd.DataFrame, pd.DataFrame]: DataFrames for the two teams playing the game.
        """
        # Ensure the 'Game ID' column is treated as a string
        self.schedule_df['Game ID'] = self.schedule_df['Game ID'].astype(str)
        
        # Check for the game ID in the schedule
        game_row = self.schedule_df.loc[self.schedule_df['Game ID'] == game_id]
        
        if game_row.empty:
            raise ValueError(f"Game ID {game_id} not found in the schedule.")
        
        game = game_row.iloc[0]
        home_team = self.teams_df[self.teams_df['abbreviation'] == game['Home Team Abbreviation']]
        away_team = self.teams_df[self.teams_df['abbreviation'] == game['Visiting Team Abbreviation']]
        
        return home_team, away_team

    def get_todays_games(self, today):
        """
        Retrieves the games scheduled for a specific date.
        Args:
            today (str): The date in 'YYYY-MM-DD' format.

        Returns:
            pd.DataFrame: DataFrame containing the games scheduled for the specified date.
        """
        if self.schedule_df is not None:
            todays_games = self.schedule_df[self.schedule_df["Game Date"] == today]
            return todays_games
        else:
            print("Schedule not loaded.")
            return None

    def get_full_schedule(self):
        """
        Returns the full schedule DataFrame.
        Returns:
            pd.DataFrame: DataFrame containing the entire schedule.
        """
        if self.schedule_df is not None:
            return self.schedule_df
        else:
            print("Schedule not loaded.")
            return None
    
    def fetch_league_standings(self, season_type):
        """
        Fetches the league standings for the specified season and season type.
        Args:
            season_type (str): The type of season ('Regular Season', 'Playoffs', etc.)

        Returns:
            pd.DataFrame: DataFrame containing the league standings.
        """
        league_standings = LeagueStandingsV3(
            league_id="00",
            season=self.season,
            season_type=season_type
        )
        standings_data = league_standings.standings.get_dict()
        self.standings_df = pd.DataFrame(standings_data['data'], columns=standings_data['headers'])
        self.standings_df.index = np.arange(1, len(self.standings_df) + 1)
        self.standings_df = self.standings_df.drop(columns=['SeasonID', 'LeagueID', 'TeamSlug', 'LeagueRank'])
        self.standings_df = self.standings_df.rename(columns={'TeamID': 'id'})
        return self.standings_df

    def process_league_standings(self, season_type):
        """
        Processes the league standings by division and conference.
        Args:
            season_type (str): The type of season ('Regular Season', 'Playoffs', etc.)

        Returns:
            pd.DataFrame: DataFrame sorted by Conference and Division.
        """
        # Fetch and process the standings
        league_standings = self.fetch_league_standings(season_type)

        # Group by 'Division' and create separate DataFrames for each division
        grouped = league_standings.groupby('Division')
        divisions = {division_name: group for division_name, group in grouped}
        
        # Prepare and modify DataFrames for each division
        modified_dfs = []
        for division, df in divisions.items():
            # Create a separator row with the division name
            separator_df = pd.DataFrame([{col: '' for col in df.columns}])
            separator_df.iloc[0, df.columns.get_loc('Division')] = division
            
            # Concatenate the separator row and the division DataFrame
            combined_df = pd.concat([separator_df, df])
            
            # Append the combined DataFrame to the list
            modified_dfs.append(combined_df)
        
        # Combine all modified DataFrames
        final_df = pd.concat(modified_dfs, ignore_index=True)
        
        # Order by 'Conference' and set specific values
        conference_order = ['East', 'West']
        final_df['Conference'] = pd.Categorical(final_df['Conference'], categories=conference_order, ordered=True)
        
        # Set specific 'Conference' values based on the provided indices
        final_df.loc[0, 'Conference'] = "East"
        final_df.loc[6, 'Conference'] = "East"
        final_df.loc[12, 'Conference'] = "West"
        final_df.loc[18, 'Conference'] = "West"
        final_df.loc[24, 'Conference'] = "East"
        final_df.loc[30, 'Conference'] = "West"
        
        # Sort the DataFrame by 'Conference' and 'Division'
        final_df_sorted = final_df.sort_values(by=['Conference', 'Division'])
        
        return final_df_sorted

    def fetch_player_career_stats(self, player_id):
        """
        Fetches career statistics for a player.
        Args:
            player_id (int): The ID of the player.

        Returns:
            pd.DataFrame: DataFrame containing the player's career statistics.
        """
        try:
            career_stats = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]
            self.player_stats[player_id] = {'career_stats': career_stats}
            return career_stats
        except Exception as e:
            print(f"Error fetching career stats for player ID {player_id}: {e}")
            return None

    def fetch_player_game_logs(self, player_id, season):
        """
        Fetches game logs for a player for a specific season.
        Args:
            player_id (int): The ID of the player.
            season (str): The NBA season in 'YYYY-YY' format, e.g., '2024-25'.

        Returns:
            pd.DataFrame: DataFrame containing the player's game logs for the season.
        """
        try:
            game_logs = playergamelog.PlayerGameLog(player_id=player_id, season=season).get_data_frames()[0]
            if player_id not in self.player_stats:
                self.player_stats[player_id] = {}
            self.player_stats[player_id]['game_logs'] = game_logs
            return game_logs
        except Exception as e:
            print(f"Error fetching game logs for player ID {player_id} in season {season}: {e}")
            return None

    def get_player_stats(self, player_id):
        """
        Retrieves the stored stats for a specific player.
        Args:
            player_id (int): The ID of the player.

        Returns:
            dict: A dictionary containing the player's career stats and game logs.
        """
        return self.player_stats.get(player_id, None)

    def compare_players(self, home_player_id, visiting_player_id, season="2023-24"):
        """
        Compares two players using the NBA API's PlayerCompare endpoint.
        Args:
            home_player_id (int): The ID of the home player.
            visiting_player_id (int): The ID of the visiting player.
            season (str): The NBA season in 'YYYY-YY' format, e.g., '2023-24'.

        Returns:
            dict: A dictionary with individual comparison data and overall comparison data.
        """
        try:
            player_compare_api = PlayerCompare(
                vs_player_id_list=str(visiting_player_id),
                player_id_list=str(home_player_id),
                season=season
            )
            
            # Retrieve the data
            individual_data = player_compare_api.individual.get_data_frame()
            overall_compare_data = player_compare_api.overall_compare.get_data_frame()
            
            return {
                "individual_data": individual_data,
                "overall_compare_data": overall_compare_data
            }
        except Exception as e:
            print(f"Error comparing players {home_player_id} and {visiting_player_id}: {e}")
            return None

    def fetch_win_probability(self, game_id, run_type='each second'):
        """
        Fetches the win probability data for a specific game using the NBA API.
        Args:
            game_id (str): The ID of the game.
            run_type (str): The granularity of the data ('each second', 'each possession').

        Returns:
            dict: A dictionary with game information and win probability data.
        """
        try:
            win_prob_api = winprobabilitypbp.WinProbabilityPBP(
                game_id=game_id,
                run_type=run_type
            )
            
            game_info = win_prob_api.game_info.get_data_frame()
            win_prob_data = win_prob_api.win_prob_pbp.get_data_frame()
            
            return {
                "game_info": game_info,
                "win_probability": win_prob_data
            }
        except Exception as e:
            print(f"Error fetching win probability data for Game ID {game_id}: {e}")
            return None

    def get_win_probability(self, game_id, run_type='each second'):
        """
        Retrieves the win probability data for a specific game.
        Args:
            game_id (str): The ID of the game.
            run_type (str): The granularity of the data ('each second', 'each possession').

        Returns:
            pd.DataFrame: DataFrame containing the win probability data.
        """
        win_prob_data = self.fetch_win_probability(game_id, run_type)
        if win_prob_data:
            return win_prob_data['win_probability']
        else:
            return None

# Roster Search

In [82]:
# Example usage
nba_rosters = NBATeamRosters(season="2024")
nba_rosters.fetch_rosters()

In [83]:
# Fetch and print the teams DataFrame
teams_df = nba_data.teams_df
teams_df

,full_name,abbreviation,nickname,city,state,year_founded
id,,,,,,
1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967


In [84]:
# Retrieve roster by abbreviation, nickname, or city
lakers_roster = nba_rosters.get_roster_by_key('LAL')
lakers_roster

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612747,2024,00,Armel Traore,Armel,armel-traore,None,F,6-8,215,"JAN 23, 2003",21.0,R,ADA Blois,1642422,Signed on 07/03/24
1,1610612747,2024,00,Blake Hinson,Blake,blake-hinson,None,F,6-8,230,"DEC 26, 1999",24.0,R,Pittsburgh,1642396,Signed on 07/03/24
2,1610612747,2024,00,Quincy Olivari,Quincy,quincy-olivari,None,None,None,None,"MAY 27, 2001",23.0,R,Xavier,1642439,Signed on 08/13/24
3,1610612747,2024,00,Kylor Kelley,Kylor,kylor-kelley,None,C,7-0,215,"AUG 26, 1997",27.0,R,Oregon State,1630283,Signed on 08/13/24
4,1610612747,2024,00,Jalen Hood-Schifino,Jalen,jalen-hood-schifino,0,G,6-5,215,"JUN 19, 2003",21.0,1,Indiana,1641720,#17 Pick in 2023 Draft
5,1610612747,2024,00,D'Angelo Russell,D'Angelo,dangelo-russell,1,G,6-3,193,"FEB 23, 1996",28.0,9,Ohio State,1626156,Traded from MIN on 02/09/23
6,1610612747,2024,00,Jarred Vanderbilt,Jarred,jarred-vanderbilt,2,F,6-8,214,"APR 03, 1999",25.0,6,Kentucky,1629020,Traded from UTA on 02/09/23
7,1610612747,2024,00,Anthony Davis,Anthony,anthony-davis,3,F-C,6-10,253,"MAR 11, 1993",31.0,12,Kentucky,203076,Traded from NOP on 07/06/19
8,1610612747,2024,00,Skylar Mays,Skylar,skylar-mays,4,G,6-3,205,"SEP 05, 1997",26.0,4,Louisiana State,1630219,Signed on 01/08/24
9,1610612747,2024,00,Dalton Knecht,Dalton,dalton-knecht,4,F,6-6,215,"APR 19, 2001",23.0,R,Tennessee,1642261,#17 Pick in 2024 Draft


In [85]:
celtics_roster = nba_rosters.get_roster_by_key('Celtics')
celtics_roster

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612738,2024,00,Baylor Scheierman,Baylor,baylor-scheierman,None,G,6-6,205,"SEP 26, 2000",23.0,R,Creighton,1631248,#30 Pick in 2024 Draft
1,1610612738,2024,00,Anton Watson,Anton,anton-watson,None,F,6-8,225,"OCT 06, 2000",23.0,R,Gonzaga,1641817,#54 Pick in 2024 Draft
2,1610612738,2024,00,Tristan Enaruna,Tristan,tristan-enaruna,None,F,6-8,220,"JUN 26, 2001",23.0,R,Cleveland State,1642400,Signed on 07/10/24
3,1610612738,2024,00,Jayson Tatum,Jayson,jayson-tatum,0,F-G,6-8,210,"MAR 03, 1998",26.0,7,Duke,1628369,#3 Pick in 2017 Draft
4,1610612738,2024,00,Jrue Holiday,Jrue,jrue-holiday,4,G,6-4,205,"JUN 12, 1990",34.0,15,UCLA,201950,Traded from POR on 10/01/23
5,1610612738,2024,00,Jaylen Brown,Jaylen,jaylen-brown,7,G-F,6-6,223,"OCT 24, 1996",27.0,8,California,1627759,#3 Pick in 2016 Draft
6,1610612738,2024,00,Kristaps Porzingis,Kristaps,kristaps-porzingis,8,F-C,7-2,240,"AUG 02, 1995",29.0,8,Cajasol Sevilla,204001,Traded from WAS on 06/23/23
7,1610612738,2024,00,Ron Harper Jr.,Ron,ron-harper-jr,8,G-F,6-4,233,"APR 12, 2000",24.0,2,Rutgers,1631199,Signed on 07/13/24
8,1610612738,2024,00,Lonnie Walker IV,Lonnie,lonnie-walker-iv,8,G-F,6-4,204,"DEC 14, 1998",25.0,6,Miami,1629022,Signed on 08/28/24
9,1610612738,2024,00,Derrick White,Derrick,derrick-white,9,G,6-4,190,"JUL 02, 1994",30.0,7,Colorado,1628401,Traded from SAS on 02/10/22


In [86]:
boston_roster = nba_rosters.get_roster_by_key('Boston')
boston_roster

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612738,2024,00,Baylor Scheierman,Baylor,baylor-scheierman,None,G,6-6,205,"SEP 26, 2000",23.0,R,Creighton,1631248,#30 Pick in 2024 Draft
1,1610612738,2024,00,Anton Watson,Anton,anton-watson,None,F,6-8,225,"OCT 06, 2000",23.0,R,Gonzaga,1641817,#54 Pick in 2024 Draft
2,1610612738,2024,00,Tristan Enaruna,Tristan,tristan-enaruna,None,F,6-8,220,"JUN 26, 2001",23.0,R,Cleveland State,1642400,Signed on 07/10/24
3,1610612738,2024,00,Jayson Tatum,Jayson,jayson-tatum,0,F-G,6-8,210,"MAR 03, 1998",26.0,7,Duke,1628369,#3 Pick in 2017 Draft
4,1610612738,2024,00,Jrue Holiday,Jrue,jrue-holiday,4,G,6-4,205,"JUN 12, 1990",34.0,15,UCLA,201950,Traded from POR on 10/01/23
5,1610612738,2024,00,Jaylen Brown,Jaylen,jaylen-brown,7,G-F,6-6,223,"OCT 24, 1996",27.0,8,California,1627759,#3 Pick in 2016 Draft
6,1610612738,2024,00,Kristaps Porzingis,Kristaps,kristaps-porzingis,8,F-C,7-2,240,"AUG 02, 1995",29.0,8,Cajasol Sevilla,204001,Traded from WAS on 06/23/23
7,1610612738,2024,00,Ron Harper Jr.,Ron,ron-harper-jr,8,G-F,6-4,233,"APR 12, 2000",24.0,2,Rutgers,1631199,Signed on 07/13/24
8,1610612738,2024,00,Lonnie Walker IV,Lonnie,lonnie-walker-iv,8,G-F,6-4,204,"DEC 14, 1998",25.0,6,Miami,1629022,Signed on 08/28/24
9,1610612738,2024,00,Derrick White,Derrick,derrick-white,9,G,6-4,190,"JUL 02, 1994",30.0,7,Colorado,1628401,Traded from SAS on 02/10/22


# Standings

In [87]:
# Instantiate the class for the 2024-2025 season
nba_data = NBATeamRosters(season="2024")

In [88]:
# Fetch and process league standings for the Regular Season
processed_standings = nba_data.process_league_standings(season_type="Regular Season")
processed_standings

,id,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,DivisionRecord,DivisionRank,WINS,LOSSES,WinPCT,Record,HOME,ROAD,L10,Last10Home,Last10Road,OT,ThreePTSOrLess,TenPTSOrMore,LongHomeStreak,strLongHomeStreak,LongRoadStreak,strLongRoadStreak,LongWinStreak,LongLossStreak,CurrentHomeStreak,strCurrentHomeStreak,CurrentRoadStreak,strCurrentRoadStreak,CurrentStreak,strCurrentStreak,ConferenceGamesBack,DivisionGamesBack,ClinchedConferenceTitle,ClinchedDivisionTitle,ClinchedPlayoffBirth,ClinchedPlayIn,EliminatedConference,EliminatedDivision,AheadAtHalf,BehindAtHalf,TiedAtHalf,AheadAtThird,BehindAtThird,TiedAtThird,Score100PTS,OppScore100PTS,OppOver500,LeadInFGPCT,LeadInReb,FewerTurnovers,PointsPG,OppPointsPG,DiffPointsPG,vsEast,vsAtlantic,vsCentral,vsSoutheast,vsWest,vsNorthwest,vsPacific,vsSouthwest,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Score_80_Plus,Opp_Score_80_Plus,Score_Below_80,Opp_Score_Below_80,TotalPoints,OppTotalPoints,DiffTotalPoints,LeagueGamesBack,PlayoffSeeding,ClinchedPostSeason
0,,,,East,,,,Atlantic,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1610612738,Boston,Celtics,East,0-0,2,,Atlantic,0-0,1,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
2,1610612751,Brooklyn,Nets,East,0-0,4,,Atlantic,0-0,2,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
3,1610612752,New York,Knicks,East,0-0,11,,Atlantic,0-0,3,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
4,1610612755,Philadelphia,76ers,East,0-0,13,,Atlantic,0-0,4,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
5,1610612761,Toronto,Raptors,East,0-0,14,,Atlantic,0-0,5,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
6,,,,East,,,,Central,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,1610612741,Chicago,Bulls,East,0-0,3,,Central,0-0,1,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
8,1610612739,Cleveland,Cavaliers,East,0-0,6,,Central,0-0,2,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
9,1610612765,Detroit,Pistons,East,0-0,7,,Central,0-0,3,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0


# Schedule

In [96]:
# Load the schedule from a CSV file
nba_data.load_schedule("nbaSchedule24.csv")

Schedule loaded: 1200 games


In [97]:
todays_games = nba_data.get_todays_games('2024-10-22')
todays_games

,Game Date,Game ID,Game Time,Arena,Arena City,Home Team Abbreviation,Home Conference,Home Division,Visiting Team Abbreviation,Visiting Conference,Visiting Division,Divisional Game,Conference Game
0,2024-10-22,22400061,7:30 pm ET,TD Garden,Boston,BOS,East,Atlantic,NYK,East,Atlantic,Yes,Yes
1,2024-10-22,22400062,10:00 pm ET,Crypto.com Arena,Los Angeles,LAL,West,Pacific,MIN,West,Northwest,No,Yes


In [ ]:
# Get the full schedule
full_schedule = nba_data.get_full_schedule()
full_schedule

# Player Data

In [ ]:
# Advanced Stats Formula

TmPoss = .5*(FGA+.475*FTA-OREB+TOV)
Off_Rtg = 100/(TmPoss+OppPoss)*Pts
Def_Rtg = 100/(TmPoss+OppPoss)*OppPts
eFG_pct = (FGM+.5*FG3)/FGA
TS_pct = Pts/(2*(FGA+.475*FTA))
ORB_pct = ORB/MP/(TmOREB+OppDREB)*TmMP/5
DRB_pct = DRB/MP/(TmDREB+OppOREB)*TmMP/5
TRB_pct = TRB/MP/(TmTRB+OppTRB)*TmMP/5

# Game Analyzer

In [99]:
home_team, away_team = nba_data.get_teams_for_game('22400061')

# Player Stats

In [94]:
# Instantiate the class for the 2024-2025 season
nba_data = NBATeamRosters(season="2024")

# Fetch career stats for a specific player (replace 'player_id' with an actual ID)
career_stats = nba_data.fetch_player_career_stats(player_id=201939)  # Example: Stephen Curry's ID
print("Career Stats:")
print(career_stats)

# Fetch game logs for the 2024-25 season
game_logs = nba_data.fetch_player_game_logs(player_id=201939, season="2024-25")
print("Game Logs for 2024-25:")
print(game_logs)

# Retrieve all stored stats for a player
player_stats = nba_data.get_player_stats(player_id=201939)
print("Stored Player Stats:")
print(player_stats)

Career Stats:
    PLAYER_ID SEASON_ID LEAGUE_ID     TEAM_ID TEAM_ABBREVIATION  PLAYER_AGE  \
0      201939   2009-10        00  1610612744               GSW        22.0   
1      201939   2010-11        00  1610612744               GSW        23.0   
2      201939   2011-12        00  1610612744               GSW        24.0   
3      201939   2012-13        00  1610612744               GSW        25.0   
4      201939   2013-14        00  1610612744               GSW        26.0   
5      201939   2014-15        00  1610612744               GSW        27.0   
6      201939   2015-16        00  1610612744               GSW        28.0   
7      201939   2016-17        00  1610612744               GSW        29.0   
8      201939   2017-18        00  1610612744               GSW        30.0   
9      201939   2018-19        00  1610612744               GSW        31.0   
10     201939   2019-20        00  1610612744               GSW        32.0   
11     201939   2020-21        00  161

In [92]:


# Suppose you have the player IDs from your rosters
home_player_id = 203957  # Example: Zach LaVine
visiting_player_id = 1628991  # Example: Jayson Tatum

# Compare the two players
comparison_results = nba_data.compare_players(home_player_id, visiting_player_id, season="2023-24")

# Access individual and overall comparison data
individual_comparison = comparison_results["individual_data"]
overall_comparison = comparison_results["overall_compare_data"]

print("Individual Comparison Data:")
print(individual_comparison)

print("Overall Comparison Data:")
print(overall_comparison)

Individual Comparison Data:
       GROUP_SET     DESCRIPTION          MIN  FGM   FGA  FG_PCT  FG3M  FG3A  \
0     INDIVIDUAL         D. Exum  1087.621667  161   302   0.533    53   108   
1  VS INDIVIDUAL  J. Jackson Jr.  2124.180000  516  1161   0.444   117   366   

   FG3_PCT  FTM  FTA  FT_PCT  OREB  DREB  REB  AST  TOV  STL  BLK  BLKA   PF  \
0    0.491   53   68   0.779    25   124  149  157   49   22    5    18   85   
1    0.320  337  417   0.808    85   280  365  154  160   80  106   121  235   

   PFD   PTS  PLUS_MINUS  
0   78   428       239.0  
1  378  1486      -340.0  
Overall Comparison Data:
    GROUP_SET                      DESCRIPTION          MIN  FGM   FGA  \
0  VS OVERALL  J. Jackson Jr.'s combined stats  2124.180000  516  1161   
1     OVERALL         D. Exum's combined stats  1087.621667  161   302   

   FG_PCT  FG3M  FG3A  FG3_PCT  FTM  FTA  FT_PCT  OREB  DREB  REB  AST  TOV  \
0   0.444   117   366    0.320  337  417   0.808    85   280  365  154  160   
1  